In [1]:
# import dependencies

from datetime import datetime as dt
import json
import numpy as np
import pandas as pd
import re

### Load in the data

In [2]:
# Read the data into a DataFrame
crowdfunding_data = pd.ExcelFile("Resources/data/crowdfunding.xlsx")
# Get the sheet names.
crowdfunding_data.sheet_names

['crowdfunding_info', 'contact_info']

In [3]:
# Get the crowdfunding_info from the crowdfunding_info worksheet. 
crowdfunding_df = pd.read_excel(crowdfunding_data, sheet_name='crowdfunding_info')
crowdfunding_df.head()

,cf_id,company_name,blurb,goal,pledged,outcome,backers_count,country,currency,launched_at,deadline,staff_pick,spotlight,category & sub-category
0,147,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100,0,failed,0,CA,CAD,1581573600,1614578400,False,False,food/food trucks
1,1621,Odom Inc,Managed bottom-line architecture,1400,14560,successful,158,US,USD,1611554400,1621918800,False,True,music/rock
2,1812,"Melton, Robinson and Fritz",Function-based leadingedge pricing structure,108400,142523,successful,1425,AU,AUD,1608184800,1640844000,False,False,technology/web
3,2156,"Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,4200,2477,failed,24,US,USD,1634792400,1642399200,False,False,music/rock
4,1365,Larson-Little,Proactive foreground core,7600,5265,failed,53,US,USD,1608530400,1629694800,False,False,theater/plays


In [4]:
# get the contact info from the crowdfunding worksheet.
contact_info_df = pd.read_excel(crowdfunding_data, sheet_name='contact_info')
contact_info_df.head()

,contact_info
0,"{""contact_id"": 4661, ""name"": ""Cecilia Velasco""..."
1,"{""contact_id"": 3765, ""name"": ""Mariana Ellis"", ..."
2,"{""contact_id"": 4187, ""name"": ""Sofie Woods"", ""e..."
3,"{""contact_id"": 4941, ""name"": ""Jeanette Iannott..."
4,"{""contact_id"": 2199, ""name"": ""Samuel Sorgatz"",..."


In [5]:
# Get the backers_info from the  updated csv. 
pd.set_option('max_colwidth', 400)
raw_backer_df = pd.read_csv('Resources/data/backer_info.csv')
raw_backer_df

,backer_info
0,"{""backer_id"": ""av166"", ""cf_id"": 968, ""name"": ""Angelo Vincent"", ""email"": ""avincent@live.com""}"
1,"{""backer_id"": ""ha127"", ""cf_id"": 563, ""name"": ""Hubert Arnold"", ""email"": ""harnold@yandex.com""}"
2,"{""backer_id"": ""lg794"", ""cf_id"": 65, ""name"": ""Loris Goulet"", ""email"": ""lgoulet@yandex.com""}"
3,"{""backer_id"": ""tb566"", ""cf_id"": 563, ""name"": ""Teodora Brunelli"", ""email"": ""tbrunelli@outlook.com""}"
4,"{""backer_id"": ""lh506"", ""cf_id"": 563, ""name"": ""Lexie Hunt"", ""email"": ""lhunt@live.com""}"
...,...
8170,"{""backer_id"": ""st581"", ""cf_id"": 65, ""name"": ""Serita Thebault"", ""email"": ""sthebault@yandex.com""}"
8171,"{""backer_id"": ""gf637"", ""cf_id"": 563, ""name"": ""Glenn Foerstner"", ""email"": ""gfoerstner@yahoo.com""}"
8172,"{""backer_id"": ""rc983"", ""cf_id"": 1114, ""name"": ""Robt Collin"", ""email"": ""rcollin@outlook.com""}"
8173,"{""backer_id"": ""cz381"", ""cf_id"": 65, ""name"": ""Corina Zappa"", ""email"": ""czappa@outlook.com""}"


### Create Database Tables

#### Contacts table

In [6]:
# Iterate through the contact_info_df and convert each row to a dictionary.
contact_values = []
for i, row in contact_info_df.iterrows():
    # Get the data first item in each row.
    data = row['contact_info']
    converted_data = json.loads(data)
    # Iterate through each dictionary (row) and get the values for each row using list comprehension.
    row_values = [v for k, v in converted_data.items()]
    # Append the list of values for each row to a new list. 
    contact_values.append(row_values)

# Create a contact_info DataFrame.
contacts_df = pd.DataFrame(contact_values, columns=['contact_id', 'name', 'email'])
contacts_df.head()

,contact_id,name,email
0,4661,Cecilia Velasco,cecilia.velasco@rodrigues.fr
1,3765,Mariana Ellis,mariana.ellis@rossi.org
2,4187,Sofie Woods,sofie.woods@riviere.com
3,4941,Jeanette Iannotti,jeanette.iannotti@yahoo.com
4,2199,Samuel Sorgatz,samuel.sorgatz@gmail.com


In [7]:
# Create a "first"name" and "last_name" column. 
contacts_df[["first_name","last_name"]] = contacts_df["name"].str.split(' ', n=1, expand=True)

# Drop the name column. 
contacts_df = contacts_df.drop(['name'], axis=1)

# re-order the columns
contacts_df = contacts_df[['contact_id', 'first_name', 'last_name', 'email']]

In [8]:
# output the contacts_df to a csv file
contacts_df.to_csv("Resources/output/contacts.csv", index=False)

#### Category and Subcategory Tables

In [9]:
# Assign the category and subcategory values to category and subcategory columns.
crowdfunding_df[["category","subcategory"]] = crowdfunding_df["category & sub-category"].str.split('/', n=1, expand=True)
crowdfunding_df.head(10)

,cf_id,company_name,blurb,goal,pledged,outcome,backers_count,country,currency,launched_at,deadline,staff_pick,spotlight,category & sub-category,category,subcategory
0,147,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100,0,failed,0,CA,CAD,1581573600,1614578400,False,False,food/food trucks,food,food trucks
1,1621,Odom Inc,Managed bottom-line architecture,1400,14560,successful,158,US,USD,1611554400,1621918800,False,True,music/rock,music,rock
2,1812,"Melton, Robinson and Fritz",Function-based leadingedge pricing structure,108400,142523,successful,1425,AU,AUD,1608184800,1640844000,False,False,technology/web,technology,web
3,2156,"Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,4200,2477,failed,24,US,USD,1634792400,1642399200,False,False,music/rock,music,rock
4,1365,Larson-Little,Proactive foreground core,7600,5265,failed,53,US,USD,1608530400,1629694800,False,False,theater/plays,theater,plays
5,2057,Harris Group,Open-source optimizing database,7600,13195,successful,174,DK,DKK,1607666400,1630213200,False,False,theater/plays,theater,plays
6,1894,"Ortiz, Coleman and Mitchell",Operative upward-trending algorithm,5200,1090,failed,18,GB,GBP,1596171600,1620709200,False,False,film & video/documentary,film & video,documentary
7,2669,Carter-Guzman,Centralized cohesive challenge,4500,14741,successful,227,DK,DKK,1608616800,1632200400,False,False,theater/plays,theater,plays
8,1114,Nunez-Richards,Exclusive attitude-oriented intranet,110100,21946,live,708,DK,DKK,1586322000,1615356000,False,False,theater/plays,theater,plays
9,970,"Rangel, Holt and Jones",Open-source fresh-thinking model,6200,3208,failed,44,US,USD,1628830800,1630386000,False,False,music/electric music,music,electric music


In [10]:
# Get the unique categories and subcategories in separate lists.
categories = crowdfunding_df["category"].unique()
subcategories = crowdfunding_df["subcategory"].unique()
print(categories)
print()
print(subcategories)

['food' 'music' 'technology' 'theater' 'film & video' 'publishing' 'games'
 'photography' 'journalism']

['food trucks' 'rock' 'web' 'plays' 'documentary' 'electric music' 'drama'
 'indie rock' 'wearables' 'nonfiction' 'animation' 'video games' 'shorts'
 'fiction' 'photography books' 'radio & podcasts' 'metal' 'jazz'
 'translations' 'television' 'mobile games' 'world music'
 'science fiction' 'audio']


In [11]:
# Create numpy arrays that have 1-10 for the category_ids and 1-25 for the subcategory_ids.
category_ids = np.arange(1, 10)
subcategory_ids = np.arange(1, 25)

# add "cat" to each category_id. 
cat_ids = ["cat" + str(cat_id) for cat_id in category_ids]

# add "scat" to each subcategory_id. 
scat_ids = ["scat" + str(scat_id) for scat_id in subcategory_ids ]

# Create the category DataFrame.
category_df = pd.DataFrame({
    "category_id": cat_ids,
    "category" : categories
})

# Create the subcategory DataFrame.
subcategory_df = pd.DataFrame({
    "subcategory_id": scat_ids,
    "subcategory" : subcategories
})

In [12]:
# verify the dfs

category_df.head(), subcategory_df.head()

(  category_id      category
 0        cat1          food
 1        cat2         music
 2        cat3    technology
 3        cat4       theater
 4        cat5  film & video,
   subcategory_id  subcategory
 0          scat1  food trucks
 1          scat2         rock
 2          scat3          web
 3          scat4        plays
 4          scat5  documentary)

In [13]:
# export the category and subcategory DataFrames to csv files.
category_df.to_csv("Resources/output/category.csv", index=False)
subcategory_df.to_csv("Resources/output/subcategory.csv", index=False)

#### Campaign Table

In [14]:
# Create a copy of the crowdfunding_info_df DataFrame name campaign_df.
campaign_df = crowdfunding_df.copy()
campaign_df.head()

,cf_id,company_name,blurb,goal,pledged,outcome,backers_count,country,currency,launched_at,deadline,staff_pick,spotlight,category & sub-category,category,subcategory
0,147,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100,0,failed,0,CA,CAD,1581573600,1614578400,False,False,food/food trucks,food,food trucks
1,1621,Odom Inc,Managed bottom-line architecture,1400,14560,successful,158,US,USD,1611554400,1621918800,False,True,music/rock,music,rock
2,1812,"Melton, Robinson and Fritz",Function-based leadingedge pricing structure,108400,142523,successful,1425,AU,AUD,1608184800,1640844000,False,False,technology/web,technology,web
3,2156,"Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,4200,2477,failed,24,US,USD,1634792400,1642399200,False,False,music/rock,music,rock
4,1365,Larson-Little,Proactive foreground core,7600,5265,failed,53,US,USD,1608530400,1629694800,False,False,theater/plays,theater,plays


In [15]:
# clean the column names
campaign_df.columns = ['cf_id', 'company_name', 'description', 'goal', 'pledged', 'outcome',
       'backers_count', 'country', 'currency', 'launch_date', 'end_date',
       'staff_pick', 'spotlight', 'category & sub-category', 'category',
       'subcategory']

In [16]:
# Convert the goal and pledged columns to a `float` data type.
campaign_df[["goal","pledged"]] = campaign_df[["goal","pledged"]].astype(float)

In [17]:
# Format the launched_date and end_date columns to datetime format.

campaign_df["launch_date"] = pd.to_datetime(campaign_df["launch_date"], unit='s').dt.strftime('%Y-%m-%d') 
campaign_df["end_date"] = pd.to_datetime(campaign_df["end_date"], unit='s').dt.strftime('%Y-%m-%d')

In [18]:
# merge the category and subcategory DataFrames with the campaign_df DataFrame.

campaign_df = campaign_df.\
    merge(category_df, on="category", how="left").\
        merge(subcategory_df, on="subcategory", how="left")

In [19]:
# extract the conract ID number from contact_df and add it to the campaign_df DataFrame.
campaign_df["contact_id"] = pd.to_numeric(contacts_df["contact_id"])

In [20]:
# Drop the unnecessary columns and reorder.
campaign_df = campaign_df.drop(columns=['staff_pick', 'spotlight', 'category & sub-category','category', 'subcategory'], axis=1)

campaign_df = campaign_df[[
                    "cf_id", 
                    "contact_id", 
                    "company_name", 
                    "description", 
                    "goal", 
                    "pledged", 
                    "outcome", 
                    "backers_count", 
                    "country", 
                    "currency", 
                    "launch_date", 
                    "end_date", 
                    "category_id",
                    "subcategory_id"
                    ]]

In [21]:
# Export the campaign_df to csv.
campaign_df.to_csv("Resources/output/campaign.csv",encoding='utf-8', index=False)

#### Backers table

In [22]:
# create a blank dataframe to hold extracted data
backer_df = pd.DataFrame()

In [23]:
# Define Regex Patterns and Extract the backer_id, cf_id, name, and email from the raw_backer_df DataFrame.

backer_id_regex = '(\w{2}\d{3})'
cf_id_regex = '":\s(\d+),\s'
name_regex = '"name":\s"(\S+\s\S+)",'
email_regex = '(\w+@\w+.\w+)'

backer_df["backer_id"] = raw_backer_df["backer_info"].str.extract(backer_id_regex)
backer_df["cf_id"] = raw_backer_df["backer_info"].str.extract(cf_id_regex)
backer_df["name"] = raw_backer_df["backer_info"].str.extract(name_regex)
backer_df["email"] = raw_backer_df["backer_info"].str.extract(email_regex)

# verify data in the new DataFrame.
backer_df

,backer_id,cf_id,name,email
0,av166,968,Angelo Vincent,avincent@live.com
1,ha127,563,Hubert Arnold,harnold@yandex.com
2,lg794,65,Loris Goulet,lgoulet@yandex.com
3,tb566,563,Teodora Brunelli,tbrunelli@outlook.com
4,lh506,563,Lexie Hunt,lhunt@live.com
...,...,...,...,...
8170,st581,65,Serita Thebault,sthebault@yandex.com
8171,gf637,563,Glenn Foerstner,gfoerstner@yahoo.com
8172,rc983,1114,Robt Collin,rcollin@outlook.com
8173,cz381,65,Corina Zappa,czappa@outlook.com


In [24]:
# Check data types.
backer_df.dtypes

backer_id    object
cf_id        object
name         object
email        object
dtype: object

In [25]:
# Convert cf_id to an integer.
backer_df['cf_id'] = pd.to_numeric(backer_df['cf_id'])

In [26]:
# Split the "name" column into "first_name" and "last_name" columns.
backer_df[['first_name','last_name']] = backer_df['name'].str.split(' ',expand=True)
backer_df

,backer_id,cf_id,name,email,first_name,last_name
0,av166,968,Angelo Vincent,avincent@live.com,Angelo,Vincent
1,ha127,563,Hubert Arnold,harnold@yandex.com,Hubert,Arnold
2,lg794,65,Loris Goulet,lgoulet@yandex.com,Loris,Goulet
3,tb566,563,Teodora Brunelli,tbrunelli@outlook.com,Teodora,Brunelli
4,lh506,563,Lexie Hunt,lhunt@live.com,Lexie,Hunt
...,...,...,...,...,...,...
8170,st581,65,Serita Thebault,sthebault@yandex.com,Serita,Thebault
8171,gf637,563,Glenn Foerstner,gfoerstner@yahoo.com,Glenn,Foerstner
8172,rc983,1114,Robt Collin,rcollin@outlook.com,Robt,Collin
8173,cz381,65,Corina Zappa,czappa@outlook.com,Corina,Zappa


In [27]:
#  Drop the name column
backer_df = backer_df.drop(columns='name')

# Reorder the columns
backer_df = backer_df[['backer_id','cf_id','first_name','last_name','email']]
backer_df

,backer_id,cf_id,first_name,last_name,email
0,av166,968,Angelo,Vincent,avincent@live.com
1,ha127,563,Hubert,Arnold,harnold@yandex.com
2,lg794,65,Loris,Goulet,lgoulet@yandex.com
3,tb566,563,Teodora,Brunelli,tbrunelli@outlook.com
4,lh506,563,Lexie,Hunt,lhunt@live.com
...,...,...,...,...,...
8170,st581,65,Serita,Thebault,sthebault@yandex.com
8171,gf637,563,Glenn,Foerstner,gfoerstner@yahoo.com
8172,rc983,1114,Robt,Collin,rcollin@outlook.com
8173,cz381,65,Corina,Zappa,czappa@outlook.com


In [28]:
# Export the DataFrame as a CSV file
backer_df.to_csv('Resources/output/backers.csv',encoding='utf8',index=False)